# Dependências

In [1]:
!pip install Owlready2

Using legacy 'setup.py install' for Owlready2, since package 'wheel' is not installed.
    Running setup.py install for Owlready2: started
    Running setup.py install for Owlready2: finished with status 'done'


You should consider upgrading via the 'C:\Users\tield\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install pyarrow

In [4]:
!pip install fastparquet

# Bibliotecas

In [5]:
from owlready2 import *
import numpy as np
import pandas as pd

# Funções

## Utilidades

In [6]:
import json
#Função para carregar descrições de amenities salvas em json
def loadJSON(fileName):
  with open(fileName) as json_file:
    itemData = json.load(json_file)
  return itemData

## Geradoras de Dados

In [7]:
def extractTuplesLayerByBin(n, wgt):

  try:
    print('Extracting tuples of bin:', n, '...')
    #Step 0: Loading the ontology
    onto_path.clear()
    ontology = get_ontology("./yelp_ontology.owl")
    ontology.load()

    #file_name = './Austin/w07/Complete bins/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'.csv'
    #tuples = pd.read_csv(file_name)
    
    file_name = './Austin/w05/Partial bins/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'-p.parquet'
    tuples = pd.read_parquet(file_name)
    #tuples = tuples[['center_poi', 'context_poi', 'distance-m']]
    
    layers = [1, 2, 3]
    for l in layers:

      layer_tuples = getTuplesFromLayer(ontology, tuples, l)

      #Se há dados
      if(len(layer_tuples) > 0):
        
        #layer_tuples_file_name = './Austin/w07/Complete bins/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'-l'+str(l)+'.csv'          
        #layer_tuples.to_csv(layer_tuples_file_name, index=False)
        
        layer_tuples_file_name = './Austin/w05/Partial bins/austin-sl-tuple-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'-l'+str(l)+'-p.parquet'          
        layer_tuples.to_parquet(layer_tuples_file_name, compression='brotli',  index = False)
                   
  except Exception as e:
    #e
    print(str(e))

In [8]:
def extractSentencesLayerByBin(n, wgt):

  try:
    print('Extracting sentences of bin:', n, '...')
    #Step 0: Loading the ontology
    onto_path.clear()
    ontology = get_ontology("file://yelp_ontology.owl")
    ontology.load()

    windows = [5, 7, 9, 11]

    for w in windows:

      sentences_window_file_name = './Austin-sb/w05/Sentences distance/austin-sl-sentences-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'-wnd'+str(w)+'-d.csv'
      #sentences_window_file_name = 'austin-sl-sentences-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'-wnd'+str(w)+'-d.csv'
      sentences = pd.read_csv(sentences_window_file_name)
      
      layers = [1, 2, 3]
      for l in layers:

        layer_sentences = getSentencesFromLayer(ontology, sentences, l)

        #Se há dados
        if(len(layer_sentences) > 0):
          layer_sentences_file_name = './Austin-sb/w05/Sentences distance/austin-sl-sentences-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'-wnd'+str(w)+'-l'+str(l)+'-d.csv'    
          #layer_sentences_file_name = 'austin-sl-sentences-n-itdl-'+str(n)+'bin-wgt'+str(wgt)+'-wnd'+str(w)+'-l'+str(l)+'-d.csv'   
          layer_sentences.to_csv(layer_sentences_file_name, index=False)
                      
  except Exception as e:
    e
    #print(str(e))

In [18]:
def getTuplesFromLayer(ontology, tuples, layer):

  layer_tuples = pd.DataFrame(columns=tuples.columns.to_list())
  for id, row in tuples.iterrows():

    centerPoI = row['center_poi'].replace(' ', '_')
    contextPoI = row['context_poi'].replace(' ', '_')

    try:

      centerPoILayer = getClassLevel(ontology, centerPoI)
      contextPoILayer = getClassLevel(ontology, contextPoI)

      
      if (centerPoILayer == contextPoILayer and layer in centerPoILayer):

        layer_tuples = layer_tuples.append(pd.DataFrame(
            [tuples.iloc[id].tolist()],
            columns=tuples.columns.to_list()), ignore_index = True
        )
    except Exception as e:
      e
      #print(str(e))
          
  return layer_tuples

In [10]:
def getSentencesFromLayer(ontology, sentences, layer):

  layer_sentences = pd.DataFrame(columns=sentences.columns.to_list())

  for id in range(0, len(sentences)):
    pois = sentences.iloc[id][sentences.iloc[id].values != 'Sentinel'].to_list()

    centerPoI = pois[0].replace(' ', '_')
    contextPoI = pois[1].replace(' ', '_')

    try:

      centerPoILayer = getClassLevel(ontology, centerPoI)
      contextPoILayer = getClassLevel(ontology, contextPoI)

      
      if (centerPoILayer == contextPoILayer and layer in centerPoILayer):

        layer_sentences = layer_sentences.append(pd.DataFrame(
            [sentences.iloc[id].tolist()],
            columns=sentences.columns.to_list()), ignore_index = True
        )
    except Exception as e:
      e
          
  return layer_sentences

In [11]:
#TODO transformar no PAI para comparação
def getRankingHitFromLevel(ontology, ranking, layer):
  layer_ranking = pd.DataFrame(columns=ranking.columns.to_list())

  for id, row in ranking.iterrows():

    type1 = row['Type1'].replace(' ', '_')
    type2 = row['Type2'].replace(' ', '_')

    try:

      type1Layer = getClassLevel(ontology, type1)
      type2Layer = getClassLevel(ontology, type2)

      print(type1, type1Layer, type2, type2Layer)
      
      if (type1Layer == type2Layer and layer in type1Layer):

        layer_ranking = layer_ranking.append(pd.DataFrame(
            [ranking.iloc[id].tolist()],
            columns=ranking.columns.to_list()), ignore_index = True
        )
    except Exception as e:
      e
          
  return layer_ranking


In [12]:
def updateRankingHitByLevel(ontology, ranking):

  layer_ranking = pd.DataFrame(columns=ranking.columns.to_list())
  for id, row in ranking.iterrows():

    type1 = row['Type1'].replace(' ', '_')
    type2 = row['Type2'].replace(' ', '_')

    try:

      type1Layer = getClassLevel(ontology, type1)
      type2Layer = getClassLevel(ontology, type2)

        
      type1Layer = type1Layer[0]
      type2Layer = type2Layer[0]

      #Obtendo relação com os pais, até se igualar ao poi em comparação

      if (type1Layer != type2Layer):
        if (type1Layer > type2Layer):

            parentsType1 = getClassParents(ontology, type1)
            for parent in parentsType1:

              
              parentLevel = getClassLevel(ontology, parent[0])
              print(type2, type2Layer, type1, type1Layer, parent[0], parentLevel[0])

              if (parentLevel[0] == type2Layer):
                parent = parent[0].replace('_', ' ')
                layer_ranking = layer_ranking.append(pd.DataFrame([[parent, row['Type2'], row['Human Mean(1-7)']]],
                                                    columns=ranking.columns.to_list()), 
                                                    ignore_index = True
                                                    )
                break

        #type2 > type1      
        else:

            parentsType2 = getClassParents(ontology, type2)
            for parent in parentsType2:

              parentLevel = getClassLevel(ontology, parent[0])
              print(type1, type1Layer, type2, type2Layer, parent[0], parentLevel[0])

              if (parentLevel[0] == type1Layer):
                parent = parent[0].replace('_', ' ')
                layer_ranking = layer_ranking.append(pd.DataFrame([[row['Type1'], parent, row['Human Mean(1-7)']]],
                                                    columns=ranking.columns.to_list()), 
                                                    ignore_index = True
                                                    )
                break
    except Exception as e:
      print(e)

  layer_ranking = pd.concat([ranking, layer_ranking])
  return layer_ranking

In [13]:
def updateBynaryHitByLevel(ontology, binary):

  binary_df = pd.DataFrame(columns=['Type1', 'Count1', 'Type2', 'Count2', 'Type3', 'Count3'])

  for key_i in binary:
    
    places = list(binary[key_i])
    
    binary_df = binary_df.append(pd.DataFrame([[places[0], binary[key_i][places[0]],
                                    places[1], binary[key_i][places[1]],
                                    places[2], binary[key_i][places[2]]]], columns=['Type1', 'Count1', 'Type2', 'Count2', 'Type3', 'Count3']),
                                  ignore_index = True)

    try:

      level_place_1 = getClassLevel(ontology, places[0].replace(' ', '_'))[0]
      level_place_2 = getClassLevel(ontology, places[1].replace(' ', '_'))[0]
      level_place_3 = getClassLevel(ontology, places[2].replace(' ', '_'))[0]

      higher_level = level_place_1
      if (level_place_2 < higher_level):
        higher_level = level_place_2
      elif (level_place_3 < higher_level):
        higher_level = level_place_3

      #print(higher_level)
      place_1 = places[0]
      place_2 = places[1]
      place_3 = places[2]

    

      #Ajustando o primeiro lugar
      if (level_place_1 > higher_level):

        place_1_parents = getClassParents(ontology, place_1.replace(' ', '_'))
        for parent in place_1_parents:
          parent_level = getClassLevel(ontology, parent[0])

          if (parent_level[0] == higher_level):
              place_1 = parent[0].replace('_', ' ')
              break

      
      #Ajustando o segundo lugar
      if (level_place_2 > higher_level):

        place_2_parents = getClassParents(ontology, place_2.replace(' ', '_'))
        for parent in place_2_parents:
          parent_level = getClassLevel(ontology, parent[0])

          if (parent_level[0] == higher_level):
              place_2 = parent[0].replace('_', ' ')
              break

      #Ajustando o segundo lugar
      if (level_place_3 > higher_level):

        place_3_parents = getClassParents(ontology, place_3.replace(' ', '_'))
        for parent in place_3_parents:
          parent_level = getClassLevel(ontology, parent[0])

          if (parent_level[0] == higher_level):
              place_3 = parent[0].replace('_', ' ')
              break

      #print(places[0], places[1], places[2])
      #print(place_1, place_2, place_3)
      #print()

      binary_df = binary_df.append(pd.DataFrame([[place_1, binary[key_i][places[0]],
                                    place_2, binary[key_i][places[1]],
                                    place_3, binary[key_i][places[2]]]], columns=['Type1', 'Count1', 'Type2', 'Count2', 'Type3', 'Count3']),
                                  ignore_index = True)
      #dic[key_i] = {place_1: binary[key_i][places[0]], place_2: binary[key_i][places[1]], place_3: binary[key_i][places[2]]}
    except Exception as e:
      print(e)

  return binary_df

In [14]:
def getRankingHitFromLevel(ontology, binary_df, layer):

  binary_layer = pd.DataFrame(columns=['Type1', 'Count1', 'Type2', 'Count2', 'Type3', 'Count3'])

  for id, row in binary_df.iterrows():

    type1 = row['Type1'].replace(' ', '_')
    type2 = row['Type2'].replace(' ', '_')
    type3 = row['Type3'].replace(' ', '_')

    try:

      type1Layer = getClassLevel(ontology, type1)
      type2Layer = getClassLevel(ontology, type2)
      type3Layer = getClassLevel(ontology, type3)

      #print(type1, type1Layer, type2, type2Layer)
      
      if (type1Layer == type2Layer and type3Layer == type1Layer and layer in type1Layer):

        binary_layer = binary_layer.append(pd.DataFrame(
            [binary_df.iloc[id].tolist()],
            columns=binary_df.columns.to_list()), ignore_index = True
        )
    except Exception as e:
      e
  
  return binary_layer

## Ontologias

In [15]:
# Recebe a ontologia e uma classe, e devolve os pais e os níveis dos pais daquela classe
# Para cada pai de um nó, encontra o novo pai e conta a profundidade
def getClassParents(ontology, class_name):
  depth = 0
  parents_list = []
  parents_queue = []
  local_parents = ontology.get_parents_of(ontology[class_name])

  #parents_queue.append([local_parents, depth])
  parents_queue.append([local_parents, depth+1])

  while(len(parents_queue) > 0):
    [parents, local_depth] = parents_queue.pop(0)
    for parent in parents:
      #print(parent)
      parents_list.append((parent.name, local_depth+1))
      local_parents = ontology.get_parents_of(ontology[parent.name])
      parents_queue.append((local_parents, (local_depth+1)))

  #Retorna lista sem valores duplicados
  return list(set([i for i in parents_list]))

In [16]:
# Recebe a ontologia e uma classe, e devolve o nível em que aquela classe está
# Para cada pai de um nó, encontra o novo pai e conta a profundidade
def getClassLevel(ontology, class_name):
  depthList = []
  parents = getClassParents(ontology, class_name)
  #print(parents)

  for parent, depth in parents:
    
    try:
      if(len(ontology.get_parents_of(ontology[parent])) == 0):
          depthList.append(depth)
          #depthList.append(depth+1)
    except Exception as e:
      e

  finalDepth = list(set(depthList))
 
  if(len(finalDepth) == 0):
    #finalDepth.append(0)
    finalDepth.append(1)
  return finalDepth 

In [1]:
# Recebe a ontologia e o nível de classes que se deseja devolver
# Para cada pai de um nó, encontra o novo pai e conta a profundidade
def getClassesFromLevel(ontology, level):
  classes = list(ontology.classes())
  levelClasses = list() #Classes desse nível
  for classe in classes:
    depths = getClassLevel(ontology, classe.name)
    for depth in depths: 
      if (depth == level):
        levelClasses.append(classe.name)

  return levelClasses

# Seleção de Camadas

In [ ]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

wgt = 0.5

# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(int(mp.cpu_count()/4))

# Step 2: `pool.apply` the `howmany_within_range()`
bins = range(0, 14)

pool.starmap(extractTuplesLayerByBin, [(n, wgt) for n in bins])
#pool.starmap(extractSentencesLayerByBin, [(n, wgt) for n in bins])

# Step 3: Don't forget to close
pool.close()
print("Process finish.")

Number of processors:  16
Extracting tuples of bin:Extracting tuples of bin:Extracting tuples of bin:Extracting tuples of bin:    1302   ...... ...

...

